In [21]:
import numpy as np
import cv2
import time


#path_model = "C:\Users\priya\Priyansha\Computer_Vision\"
path_model = "C:/Users/priya/Priyansha/Computer_Vision/"

# Read Network
#model_name = "model-f6b98070.onnx"; # MiDaS v2.1 Large
model_name = "model-small.onnx"; # MiDaS v2.1 Small


# Load the DNN model
model = cv2.dnn.readNet(path_model + model_name)


if (model.empty()):
    print("Could not load the neural net! - Check path")


# Set backend and target to CUDA to use GPU
model.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
model.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
 
# Webcam
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(1) #to take video feed from droid cam

while cap.isOpened():

    # Read in the image
    success, img = cap.read()

    imgHeight, imgWidth, channels = img.shape

    # start time to calculate FPS
    start = time.time()


    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    
    # Create Blob from Input Image
    # MiDaS v2.1 Large ( Scale : 1 / 255, Size : 384 x 384, Mean Subtraction : ( 123.675, 116.28, 103.53 ), Channels Order : RGB )
    #blob = cv2.dnn.blobFromImage(img, 1/255., (384,384), (123.675, 116.28, 103.53), True, False)

    # MiDaS v2.1 Small ( Scale : 1 / 255, Size : 256 x 256, Mean Subtraction : ( 123.675, 116.28, 103.53 ), Channels Order : RGB )
    blob = cv2.dnn.blobFromImage(img, 1/255., (256,256), (123.675, 116.28, 103.53), True, False)

    # Set input to the model
    model.setInput(blob)

    # Make forward pass in model
    output = model.forward()
    
    output = output[0,:,:]
    output = cv2.resize(output, (imgWidth, imgHeight))

    # Normalize the output
    output = cv2.normalize(output, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Calculate the average depth in the center region of the image
    center_depth = np.mean(output[imgHeight // 4:3 * imgHeight // 4, imgWidth // 4:3 * imgWidth // 4])
    right_depth =  np.mean(output[imgHeight // 4:3 * imgHeight // 4, imgWidth // 8: imgWidth // 4])
    left_depth = np.mean(output[imgHeight // 4:3 * imgHeight // 4, 3 * imgWidth // 4 :imgWidth])
    l = [center_depth, right_depth, left_depth]
    print(l)
    # Define a threshold to determine if there is an obstacle
    obstacle_threshold = 0.25  # You may need to adjust this based on your specific environment

    # Determine the direction to avoid collision
    # if center_depth < obstacle_threshold:
    #     direction = "Move Forward"
    # else:
    #     # You may need to adjust the following logic based on your camera setup and orientation
    #     if np.mean(output[:, imgWidth // 2]) > obstacle_threshold:
    #         direction = "Turn Left"
    #     elif np.mean(output[:, imgWidth // 2]) < obstacle_threshold:
    #         direction = "Turn Right"
    #     else:
    #         direction = "Move Forward"

    if center_depth < obstacle_threshold:
         direction = "Forward"
    else:
        if center_depth == max(center_depth, right_depth, left_depth):
            direction = "Forward"
        if right_depth == max(center_depth, right_depth, left_depth):
            direction = "Right"
        elif left_depth == max(center_depth, right_depth, left_depth):
            direction = "Left"
        #else:
            direction = "Forward"

    # Display the direction
    cv2.putText(img, direction, (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Apply color map for visualization
    output_colormap = cv2.applyColorMap((output * 255).astype(np.uint8), cv2.COLORMAP_JET)

    # Convert depth map to three channels
    output_colormap = cv2.cvtColor(output_colormap, cv2.COLOR_BGR2RGB)

    # End time
    end = time.time()
    # calculate the FPS for current frame detection
    fps = 1 / (end-start)
    # Show FPS
    cv2.putText(img, f"{fps:.2f} FPS", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    cv2.imshow('image', img)
    cv2.imshow('Depth Map', output_colormap)


    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

 
cap.release()
cv2.destroyAllWindows()

[0.31231812, 0.3225544, 0.29688102]
[0.32911718, 0.3662358, 0.2978496]
[0.4258404, 0.43259856, 0.37372008]
[0.43270624, 0.38164717, 0.43329284]
[0.4476443, 0.5351033, 0.28737515]
[0.5269247, 0.6047168, 0.35481688]
[0.19795157, 0.30208382, 0.22800744]
[0.1998023, 0.29114425, 0.23689774]
[0.23180361, 0.16640098, 0.5047495]
[0.25958937, 0.24204035, 0.3675796]
[0.25958937, 0.24204035, 0.3675796]
[0.2649858, 0.2414287, 0.35031152]
[0.2649858, 0.2414287, 0.35031152]
[0.23788269, 0.17548104, 0.17161943]
[0.33110026, 0.18471646, 0.1744588]
[0.29339314, 0.12775944, 0.19333632]
[0.29239646, 0.13123593, 0.2549892]
[0.29512697, 0.18081121, 0.210701]
[0.3096453, 0.18176769, 0.22917251]
[0.34999597, 0.21270102, 0.22976238]
[0.29121253, 0.21113704, 0.11828451]
[0.29985487, 0.23482981, 0.08513367]
[0.2773128, 0.2663246, 0.093778774]
[0.25722155, 0.25008747, 0.080774985]
[0.2510204, 0.2564747, 0.07712134]
[0.24073939, 0.23356797, 0.09467057]
[0.25600028, 0.2585964, 0.10876134]
[0.27844805, 0.2629751, 0